In [107]:
import sys

sys.path.append("/home/olek/Documents/dev/metadrive-multifidelity-data/notebooks")
from utils.parse_metadrive import get_scenarios_df, process_scenario_df  # type: ignore


from utils.scenario_runner import ScenarioRunner
from utils.bayesian_optimisation import (
    SEARCH_FIDELITIES,
    BAYESOPT_INITIALIZATION_RATIO,
    SEARCH_TYPES,
    do_search,
    logger,
    FIDELITY_RANGE,
    preprocess_features,
    get_random_scenario_seed,
    get_candidate_solutions,
    regression_pipeline,
    get_mean_and_std_from_model,
    get_next_scenario_seed_from_aq,
    expected_improvement,
    upper_confidence_bound,
    pick_next_fidelity,
    DEFAULT_SEARCH_BUDGET,
    HDD_PATH,
    get_training_data,
    random_search_iteration,
    set_seed,
)
from pathlib import Path
import shutil
from itertools import count

## Bayes Opt Iteration


In [ ]:
def bayes_opt_iteration_new(
    train_df, aq_type="ei", fidelity="multifidelity"
) -> tuple[int, int]:
    """
    Performs a single iteration of Bayesian Otpimisation
    Returns next scenario seed, and next fidelity to run.

    """

    logger.info(f"Entering Bayesian Opt Iteration with parameters:")
    logger.info(f"N training samples {len(train_df)}, {aq_type = }, {fidelity = }")
    target_fidelity = fidelity
    if fidelity == "multifidelity":
        target_fidelity = max(FIDELITY_RANGE)

    # PREPARE TRAINING DATA
    X_train = preprocess_features(train_df)
    y_train = train_df["eval.driving_score"]

    if target_fidelity not in train_df.index.get_level_values("fid.ads_fps"):
        logger.warning(f"Target fidelity is not present in training set.")
        logger.warning(f"Will run target fidelity now!")
        return get_random_scenario_seed(get_candidate_solutions()), target_fidelity

    current_best = y_train.xs(target_fidelity).min()
    logger.info(f"Current best score is: {current_best:.3f}")

    # TRAIN THE MODEL
    pipe = regression_pipeline(X_train)
    logger.info(f"Training using {len(X_train.columns)} features")
    # pipe.set_params(regressor__n_jobs=16)
    model = pipe.fit(X_train, y_train)
    logger.debug(f"Model trained")

    # PREPARE TEST DATA
    candidate_scenarios = get_candidate_solutions()

    # TODO change that allow, repeating same scenario, if in diffrent fidelity
    # Exclude scenarios that have been evaluated (in any fidelity)
    if fidelity == "multifidelity":
        # Exclude scenarios that have been evaluated (in all fidelity settings)
        seeds_to_exclude = [
            seed
            for seed, df in train_df.groupby("def.seed")
            if set(df["fid.ads_fps"].unique()) == set(FIDELITY_RANGE)
        ]
    else:
        # Exclude scenarios that have been evaluated
        seeds_to_exclude = train_df["def.seed"].unique()

    logger.info(
        f"Excluding {len(seeds_to_exclude)} seeds from candidates {seeds_to_exclude = }"
    )
    candidate_scenarios = candidate_scenarios[
        ~candidate_scenarios.index.isin(seeds_to_exclude)
    ]
    logger.info(f"Considering next scenario from {len(candidate_scenarios)} candidates.")

    X_test = preprocess_features(candidate_scenarios)
    # test candidates must be casted to target fidelity
    X_test["fid.ads_fps"] = target_fidelity
    X_test = X_test[X_train.columns]

    # PREDICT DSCORE FOR HIGHFIDELITY
    dscore_predictions, std = get_mean_and_std_from_model(model, X_test)
    logger.info(f"Best from model: {dscore_predictions.min():.3f}")

    match aq_type:
        case "ei":
            aq = expected_improvement(dscore_predictions, std, current_best)
        case "ucb":
            aq = upper_confidence_bound(dscore_predictions, std)
        case _:
            raise ValueError("Invalid acquisition function")

    next_seed = int(get_next_scenario_seed_from_aq(aq, candidate_scenarios))
    logger.info(f"Next seed to evaluate: {next_seed}")

    if fidelity != "multifidelity":
        return next_seed, target_fidelity

    logger.debug(f"Multifidelity enabled")

    next_cadidate = candidate_scenarios.loc[[next_seed]]
    next_fidelity = pick_next_fidelity(next_cadidate, X_train.columns, model)

    used_fidelities = train_df[train_df["def.seed"] == next_seed]["fid.ads_fps"].unique()
    available_fidelities = set(FIDELITY_RANGE) - set(used_fidelities)

    if next_fidelity not in available_fidelities:
        logger.warning(f"Next fidelity {next_fidelity = } is not available.")
        next_fidelity = max(available_fidelities)
        logger.info(f"Fallbacking to highest avaiable: {next_fidelity} instead")

    return next_seed, next_fidelity

In [109]:
p = Path(
    "/home/olek/Documents/dev/metadrive-multifidelity-data/data/experiments/allow_repeat"
)
rep_path = p / "searches" / "bayesopt_ucb" / "multifidelity" / "13"


assert rep_path.exists()
train_df = get_training_data(rep_path=rep_path)


train_df

[2025-05-03 17:28:13,761] [MainProcess] [bayesian_optimisation.py:87] [INFO]: Loading search data from /home/olek/Documents/dev/metadrive-multifidelity-data/data/experiments/allow_repeat/searches/bayesopt_ucb/multifidelity/13
100%|██████████| 556/556 [00:00<00:00, 607142.16it/s]


,fid.ads_fps,fid.world_fps,def.seed,def.spawn_lane_index,def.distance,def.max_steps,time.init_time,time.agent_time,time.scenario_time,time.closing_time,...,def.vehicles_data.vehicle_27_position_y,def.vehicles_data.vehicle_27_position_z,def.vehicles_data.vehicle_27_type,def.vehicles_data.vehicle_27_heading_theta,def.vehicles_data.vehicle_27_length,def.vehicles_data.vehicle_27_width,def.vehicles_data.vehicle_27_height,def.vehicles_data.vehicle_27_spawn_road,def.vehicles_data.vehicle_27_destination,eval.driving_score
0,20,60,1009867,1,555.480144,16664,0.232783,5.841630,19.076307,0.394378,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
1,60,60,1020441,0,579.037529,17371,0.279412,18.869913,35.362247,0.539388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2,60,60,1009043,0,447.948324,13438,0.211925,15.179535,32.027789,0.311157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
3,10,60,1038424,0,592.936285,17788,0.303009,3.271592,19.107200,0.520760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
4,60,60,1041605,0,425.152776,12755,0.252188,49.005043,70.800144,0.403877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.120901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,10,60,1041605,0,425.152776,12755,0.234501,8.190496,24.066906,1.493648,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121737
552,10,60,1005070,1,560.296594,16809,0.317511,2.951648,16.785626,0.564337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600000
553,10,60,1041605,0,425.152776,12755,0.227855,8.480880,24.423342,0.605440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121737
554,10,60,1041605,0,425.152776,12755,0.228067,8.602958,25.410724,0.869197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121737


In [110]:
import pandas as pd

# Test dataframe where some seeds have all fidelities present
test_df_1 = pd.DataFrame(
    {
        "def.seed": [
            1000000,
            1000000,
            1000000,
            1000000,
            1000001,
            1000001,
            1000003,
            1000003,
            1000003,
            1000003,
            1000003,
        ],
        "fid.ads_fps": [60, 10, 20, 30, 10, 60, 10, 20, 20, 30, 60],
        "eval.driving_score": [0.8, 0.9, 0.7, 0.85, 0.7, 0.75, 0.6, 0.65, 0.7, 0.75, 0.8],
    }
)


seeds_to_exclude = [
    seed
    for seed, df in test_df_1.groupby("def.seed")
    if set(df["fid.ads_fps"].unique()) == set(FIDELITY_RANGE)
]
seeds_to_exclude

[1000000, 1000003]

In [111]:
candidate_scenarios = get_candidate_solutions()
candidate_scenarios = candidate_scenarios.loc[:1000004]
# candidate_scenarios

In [112]:
train_df = test_df_1.copy()
using_multifidelity = True
if using_multifidelity:
    # Exclude scenarios that have been evaluated (in all fidelity settings)
    seeds_to_exclude = [
        seed
        for seed, df in train_df.groupby("def.seed")
        if set(df["fid.ads_fps"].unique()) == set(FIDELITY_RANGE)
    ]
else:
    # Exclude scenarios that have been evaluated
    seeds_to_exclude = train_df["def.seed"].unique()


candidate_scenarios = candidate_scenarios[~candidate_scenarios.index.isin(seeds_to_exclude)]
candidate_scenarios

,fid.ads_fps,fid.world_fps,def.spawn_lane_index,def.distance,def.max_steps,time.init_time,time.agent_time,time.scenario_time,time.closing_time,def.map_seq.0.id,...,def.vehicles_data.vehicle_36_position_x,def.vehicles_data.vehicle_36_position_y,def.vehicles_data.vehicle_36_position_z,def.vehicles_data.vehicle_36_type,def.vehicles_data.vehicle_36_heading_theta,def.vehicles_data.vehicle_36_length,def.vehicles_data.vehicle_36_width,def.vehicles_data.vehicle_36_height,def.vehicles_data.vehicle_36_spawn_road,def.vehicles_data.vehicle_36_destination
def.seed,,,,,,,,,,,,,,,,,,,,,
1000001,60,60,0,615.214675,18456,0.391575,28.178873,50.854427,0.233320,I,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None
1000002,60,60,1,500.323549,15010,0.471720,24.169583,46.084278,0.203616,I,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None
1000004,60,60,1,644.202527,19326,0.535260,31.104951,84.788868,0.337844,I,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None


In [114]:
next_seed = 1000001
next_cadidate = candidate_scenarios.loc[[next_seed]]

train_df

,def.seed,fid.ads_fps,eval.driving_score
0,1000000,60,0.80
1,1000000,10,0.90
2,1000000,20,0.70
3,1000000,30,0.85
4,1000001,10,0.70
5,1000001,60,0.75
6,1000003,10,0.60
7,1000003,20,0.65
8,1000003,20,0.70
9,1000003,30,0.75


[2025-05-03 17:30:29,234] [MainProcess] [3729331500.py:7] [WARNING]: Next fidelity next_fid = 10 is not available.
[2025-05-03 17:30:29,236] [MainProcess] [3729331500.py:9] [INFO]: Fallbacking to highest avaiable: 30 instead


In [89]:
print("Test Case 1 - Seeds to Exclude:", seeds_to_exclude)
FIDELITY_RANGE

Test Case 1 - Seeds to Exclude: [1000000, 1000003]


[10, 20, 30, 60]

In [90]:
next_seed, next_fid = bayes_opt_iteration_new(train_df, "ucb", "multifidelity")
next_seed, next_fid

[2025-05-03 17:24:12,040] [MainProcess] [2827295328.py:10] [INFO]: Entering Bayesian Opt Iteration with parameters:
[2025-05-03 17:24:12,041] [MainProcess] [2827295328.py:11] [INFO]: N training samples 11, aq_type = 'ucb', fidelity = 'multifidelity'


KeyError: 'Level fid.ads_fps not found'

## Do search


In [ ]:
# This function should run in separate process


def do_search_new(
    repetition,
    search_type="randomsearch",
    fidelity="multifidelity",
    smoketest=False,
    search_root_dir=HDD_PATH,
):

    SEARCH_DIR = Path(search_root_dir) / ("searches_smoketest" if smoketest else "searches")
    SEARCH_DIR.mkdir(exist_ok=True, parents=True)

    rep_path = SEARCH_DIR / search_type / str(fidelity) / str(repetition)
    if rep_path.exists():
        logger.info(f"Search already finished for {rep_path}, skipping")
        return

    logger.info(
        f"Starting {"smoke" if smoketest else "real"} {search_type} search for: {repetition = } in {fidelity = }"
    )

    # set random seed from rep and search type
    set_seed(repetition, search_type, fidelity)

    # Initialize the search budget
    SEARCH_BUDGET = 15 if smoketest else DEFAULT_SEARCH_BUDGET
    logger.info(f"Search budget: {SEARCH_BUDGET}")
    current_budget = SEARCH_BUDGET

    for it in count():
        logger.info(f"Starting iteration {it = }")

        match search_type.split("_"):
            case ["randomsearch"]:
                logger.info("Random search iteration!")
                next_seed, next_fid = random_search_iteration(fidelity)

            case ["bayesopt", aq_type]:
                logger.info(f"{aq_type.upper()} Baysian optimisation iteration")
                if current_budget > BAYESOPT_INITIALIZATION_RATIO * SEARCH_BUDGET:
                    logger.info(f"Still initializing BayesOpt, using RS iteration")
                    next_seed, next_fid = random_search_iteration(fidelity)
                else:
                    logger.info(f"Doing BayesOpt iteration")
                    train_df = get_training_data(rep_path=rep_path)
                    next_seed, next_fid = bayes_opt_iteration_new(
                        train_df, aq_type, fidelity
                    )
            case _:
                raise ValueError(f"Invalid search type: {search_type}")

        logger.info(f"Next seed: {next_seed}, fidelity: {next_fid}")
        it_path = rep_path / str(it)
        runner = ScenarioRunner(it_path, next_seed, next_fid)
        runner.run_scenario(repeat=True)
        cost = runner.get_evaluation_cost()
        del runner

        logger.info(f"Running this scenario cost: {cost}")
        current_budget -= cost

        logger.info(f"Current budget: {current_budget}")

        if current_budget <= 0:
            logger.info(f"Budget finished!")
            with open(SEARCH_DIR / "checkpoints.txt", "a") as file:
                file.write(f"Search of {rep_path} finished successfully!\n")

            break

In [ ]:
search_dir = Path("/tmp/pipeline_check")
shutil.rmtree(search_dir, ignore_errors=True)

In [ ]:
rep = 0


do_search_new(
    rep,
    search_type="bayesopt_ucb",
    fidelity="multifidelity",
    search_root_dir=search_dir,
    smoketest=False,
)

[2025-04-22 15:16:29,041] [MainProcess] [925087489.py:20] [INFO]: Starting real bayesopt_ucb search for: repetition = 0 in fidelity = 'multifidelity'
[2025-04-22 15:16:29,041] [MainProcess] [bayesian_optimisation.py:52] [INFO]: Setting a random seed: 5030000
[2025-04-22 15:16:29,041] [MainProcess] [925087489.py:29] [INFO]: Search budget: 600
[2025-04-22 15:16:29,041] [MainProcess] [925087489.py:33] [INFO]: Starting iteration it = 0
[2025-04-22 15:16:29,042] [MainProcess] [925087489.py:41] [INFO]: UCB Baysian optimisation iteration
[2025-04-22 15:16:29,042] [MainProcess] [925087489.py:43] [INFO]: Still initializing BayesOpt, using RS iteration
[2025-04-22 15:16:30,000] [MainProcess] [925087489.py:54] [INFO]: Next seed: 1008311, fidelity: 10
[2025-04-22 15:16:30,002] [MainProcess] [scenario_runner.py:60] [INFO]: Saving data to /tmp/pipeline_check/searches/bayesopt_ucb/multifidelity/0/0
[2025-04-22 15:16:30,219] [MainProcess] [scenario_runner.py:236] [INFO]: Launching the scenario with re

In [ ]:
rep_path = Path("/tmp/pipeline_check/searches_smoketest/bayesopt_ucb/multifidelity/0")
train_df = get_training_data(rep_path=rep_path)
candidate_scenarios = get_candidate_solutions()

# TODO change that allow, repeating same scenario, if in diffrent fidelity
# Exclude scenarios that have been evaluated (in any fidelity)
# candidate_scenarios = candidate_scenarios[
#     ~candidate_scenarios.index.isin(train_df.index.get_level_values("def.seed"))
# ]

NameError: name 'Path' is not defined

In [ ]:
df = get_scenarios_df(rep_path, multiprocessed=False)
df = process_scenario_df(df)
df = df.set_index(["fid.ads_fps", "def.seed"])
df["eval.driving_score"].to_frame()

100%|██████████| 6/6 [00:00<00:00, 102717.65it/s]


eval.driving_score
fid.ads_fps def.seed                    
10          1008311             0.698156
30          1035219             0.600000
60          1030611             1.000000
10          1035219             1.000000
30          1035219             0.600000
            1054067             1.000000

In [ ]:
train_df["eval.driving_score"]

fid.ads_fps  def.seed
10           1008311     0.698156
             1035219     1.000000
30           1035219     0.600000
             1035219     0.600000
             1054067     1.000000
60           1030611     1.000000
Name: eval.driving_score, dtype: float64

In [ ]:
train_df

fid.world_fps  def.spawn_lane_index  def.distance  \
fid.ads_fps def.seed                                                      
10          1008311              60                     0    496.950598   
            1035219              60                     0    436.170788   
30          1035219              60                     0    436.170788   
            1035219              60                     0    436.170788   
            1054067              60                     2    481.226272   
60          1030611              60                     0    557.306559   

                      def.max_steps  time.init_time  time.agent_time  \
fid.ads_fps def.seed                                                   
10          1008311           14909        0.204075         7.342530   
            1035219           13085        0.131239         1.607165   
30          1035219           13085        0.131292         4.771381   
            1035219           13085        0.132042         4.741959   
            1054067           14437        0.137880         5.289513   
60          1030611           16719        0.110365        11.759548   

                      time.scenario_time  time.closing_time  \
fid.ads_fps def.seed                                          
10          1008311            31.873411           0.172453   
            1035219             8.949169           0.154206   
30          1035219            12.278898           0.149782   
            1035219            12.171943           0.151437   
            1054067            11.697764           0.147312   
60          1030611            16.383089           0.127387   

                      eval.n_crashed_vehicles  eval.termination.timeout  ...  \
fid.ads_fps def.seed                                                     ...   
10          1008311                         0                      True  ...   
            1035219                         0                     False  ...   
30          1035219                         1                     False  ...   
            1035219                         1                     False  ...   
            1054067                         0                     False  ...   
60          1030611                         0                     False  ...   

                      def.vehicles_data.vehicle_16_position_y  \
fid.ads_fps def.seed                                            
10          1008311                                154.558487   
            1035219                                       NaN   
30          1035219                                       NaN   
            1035219                                       NaN   
            1054067                                       NaN   
60          1030611                                       NaN   

                      def.vehicles_data.vehicle_16_position_z  \
fid.ads_fps def.seed                                            
10          1008311                                       1.4   
            1035219                                       NaN   
30          1035219                                       NaN   
            1035219                                       NaN   
            1054067                                       NaN   
60          1030611                                       NaN   

                      def.vehicles_data.vehicle_16_type  \
fid.ads_fps def.seed                                      
10          1008311                           XLVehicle   
            1035219                                 NaN   
30          1035219                                 NaN   
            1035219                                 NaN   
            1054067                                 NaN   
60          1030611                                 NaN   

                      def.vehicles_data.vehicle_16_heading_theta  \
fid.ads_fps def.seed                                               
10          1008311                            